In [1]:
print("Hello staya")

Hello staya


In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

/opt/conda/envs/rag_env/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
hf_key = os.getenv("huggingface_api_key")
groq_key = os.getenv("groq_api_key")

In [4]:
def load_pdf(data):
    loader = DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [5]:
extracted_data = load_pdf("data/")

In [6]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)

In [8]:
text_chunks

[Document(metadata={'source': 'data/Mahabharata.pdf', 'page': 0}, page_content="The Complete Mahabharata in English  \n \n \nThe Mahabharata  \nof \nKrishna -Dwaipayana Vyasa  \n \nBOOK 1  \nADI PARVA  \n \n \n \nTranslated into English Prose from the Original Sanskrit Text  by Kisari Mohan Ganguli  [1883 -1896]  \n \nScanned at sacred -texts.com, 2003. Proofed at Distribu ted Proofing,  Juliet Sutherland, Project Manager. Additional proofing \nand formatting at  sacred -texts.com, by J. B. Hare.  \n \nTRANSLATOR'S PREFACE  \nThe object of a translator should ever be to hold the mirror upto his  author. That being so, his chief duty is to  represent so far as  \npracticable the manner in which his author's ideas have been expressed,  retaining if possible at the sacrifice of idiom and taste \nall the  peculiarities of his author's imagery and of language as well. In regard  to translations from the  Sanskrit, nothing is easier \nthan to dish up  Hindu ideas, so as to make them agreeabl

In [9]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embedding_model = download_hugging_face_embeddings()

/tmp/ipykernel_26271/4238859041.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


/opt/conda/envs/rag_env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/conda/envs/rag_env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
query_result = embedding_model.embed_query("Hello Satya")
print("Length = ", len(query_result))

Length =  384


In [12]:
from langchain_community.vectorstores import Chroma
vector_store = Chroma.from_documents(text_chunks,embedding_model)

In [13]:
from langchain_groq import ChatGroq
model_name = "llama-3.1-70b-versatile"
llm = ChatGroq(api_key=groq_key,model=model_name)

In [19]:
from langchain.chains import RetrievalQA
rag_qa_pipeline = RetrievalQA.from_chain_type(
    llm = llm,
    retriever=vector_store.as_retriever(),
    chain_type = "stuff",
    return_source_documents=True
)

In [20]:
query = "Who was arjuna?"
answer = rag_qa_pipeline.invoke(query)
print(f"Answer: {answer["result"]}")

Answer: Arjuna was a major character in the ancient Indian epic, the Mahabharata. He was a prince of the Pandava dynasty and the third of the five Pandava brothers. Arjuna was known for his exceptional bravery, archery skills, and unwavering commitment to his duty as a warrior. He was also a close friend and devotee of Lord Krishna, who served as his charioteer and advisor during the epic battle of Kurukshetra. Arjuna is often referred to as one of the greatest heroes of the Mahabharata and is revered as a symbol of martial valor and righteousness.


In [22]:
query = "What is the Mahabharata about?"
answer = rag_qa_pipeline.invoke(query)
print(f"Answer: {answer["result"]}")

Answer: The Mahabharata is an ancient Hindu epic that tells the story of a great war between two groups of cousins, the Pandavas and the Kauravas, who are descended from the same ancestor. The epic explores themes of duty, morality, and the nature of reality, as well as the struggle between good and evil.

The story revolves around the five Pandava brothers, who are the rightful heirs to the throne of Hastinapura, and their cousins, the Kauravas, who seek to usurp the throne. The epic follows the events leading up to the war, the war itself, and its aftermath.

Throughout the epic, various characters, including the god Krishna, who serves as a mentor and guide to the Pandavas, grapple with complex moral and philosophical questions. The Mahabharata also contains numerous subplots, including romances, adventures, and philosophical debates.

Ultimately, the Mahabharata is a rich and complex work that explores many aspects of human nature and the human condition, offering insights into the

In [23]:
query = "What are the key themes in the Mahabharata?"
answer = rag_qa_pipeline.invoke(query)
print(f"Answer: {answer["result"]}")

Answer: The Mahabharata is an epic Indian poem that contains various key themes, including:

1. **Dharma** (Duty): The concept of dharma is central to the Mahabharata. It refers to an individual's duty, morality, and righteousness. The epic explores the complexities of dharma and its application in different situations.
2. **Karma** (Action and Consequences): The Mahabharata emphasizes the importance of karma, which holds that an individual's actions have consequences. The epic illustrates how the actions of characters, both good and bad, lead to their respective fates.
3. **Moksha** (Liberation): The Mahabharata explores the concept of moksha, or liberation from the cycle of birth and death. The epic presents various paths to achieve moksha, including devotion, knowledge, and self-realization.
4. **War and Violence**: The Mahabharata contains detailed descriptions of war and violence, which serve as a backdrop for exploring moral and philosophical themes. The epic examines the consequ

In [24]:
query = "What role did Krishna play in the Mahabharata?"
answer = rag_qa_pipeline.invoke(query)
print(f"Answer: {answer["result"]}")

Answer: Krishna played a significant role in the Mahabharata. He was the cousin of both the Pandavas and the Kauravas, but he was particularly close to the Pandavas, especially Arjuna. Krishna served as the charioteer of Arjuna during the Kurukshetra War, which was the central conflict of the Mahabharata.

Krishna was not only a skilled charioteer but also a wise counselor and a strategist. He provided guidance and advice to the Pandavas throughout the war, often helping them to outmaneuver their enemies.

One of the most famous episodes involving Krishna in the Mahabharata is the Bhagavad Gita, in which he delivers a philosophical sermon to Arjuna on the eve of the war. In this sermon, Krishna teaches Arjuna about the nature of duty, morality, and spiritual growth, and helps him to overcome his doubts and fears about fighting in the war.

Krishna's role in the Mahabharata was not limited to his military and strategic contributions. He was also a symbol of divine wisdom and guidance, a

In [16]:
while True:
    query=input(f"Input Prompt:")
    result=rag_qa_pipeline.invoke(query)
    print("Answer : ", result["result"])

Answer :  The Pandavas and Kauravas fought in the epic battle of Kurukshetra, as described in the Mahabharata. The main reason for this battle was a dispute over the throne of Hastinapura, the kingdom ruled by the Kauravas.

The story goes that the Pandavas, five brothers (Yudhishthira, Bhima, Arjuna, Nakula, and Sahadeva), were the sons of King Pandu, the rightful ruler of Hastinapura. However, due to a curse, Pandu was forced to abdicate the throne, and his brother, Dhritarashtra, who was blind, took over as the ruler.

Dhritarashtra had 100 sons, known as the Kauravas, with the eldest son, Duryodhana, being the most prominent. Duryodhana was jealous of the Pandavas and feared that they would take over the throne. He conspired against them and tried to kill them several times, but they always managed to escape.

Eventually, the Pandavas were forced to flee to the forest and live in exile for 13 years. When they returned, they demanded their rightful share of the kingdom, but Duryodha

KeyboardInterrupt: Interrupted by user